<a href="https://colab.research.google.com/github/mhsizar/grover-algorithm/blob/main/Grover'sAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary modules

!pip install qiskit qiskit-aer qiskit-ibm-provider numpy matplotlib

In [ ]:
# Call the libraries

from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import Aer
from qiskit_ibm_provider import IBMProvider, least_busy
from qiskit.visualization import plot_histogram, circuit_drawer
import numpy as np
import matplotlib.pyplot as plt
import time

<ipython-input-3-73906bce1328>:5: DeprecationWarning: The package qiskit_ibm_provider is being deprecated. Please see https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime to get instructions on how to migrate to qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  from qiskit_ibm_provider import IBMProvider, least_busy


## Classical Approach

In classical search algorithm, the average search is N/2. Or in the worst case N. In this code we do a simple search problem of target = 15 from a list of 16 numbers. In the code output we can see the code's runtime, and number of iterations.


In [ ]:
# Define the classical search function:
def classical_search(elements, target):
  start_time = time.time() # Storing the beginning time to calculate runtime
  iterations = 0
  for index, element in enumerate(elements):
    iterations +=1 # Increase the number of iteration at each step
    if element == target:
      end_time = time.time() # Stroring the search completion time to calculate runtime
      return index, end_time - start_time, iterations
  end_time = time.time()  # Stroing the end time of the fucntion call if the target is not found
  return -1, end_time - start_time, iterations

# Create a list of 16 elements and set the target element
elements = [i for i in range(16)]
target = 15

# Perform the classical search approach
index, classical_runtime, classical_iterations = classical_search(elements, target)

# Print the result
print(f"Classical Search: Found target = {target} at index {index} in {classical_runtime} seconds after {classical_iterations} iterations")

Classical Search: Found target = 15 at index 15 in 3.5762786865234375e-06 seconds after 16 iterations


## Quantum Approach

In this section we try to solve the same search problem using Grover's algorithm. First, we implement the code using the QAS simulator from Qiskit. In the second implementation, we use the IBM Q API to run the code in a remote IBM quantum computer.

In [ ]:
# Define the oracle for the target state |1111> (binary quivalent of 15)
def grover_oracle():
  '''Oracle is a quantum circuit that marks a specific state as the solution to the search algorithm. This function defines the state |1111> as the solution'''
  oracle = QuantumCircuit(4, name='oracle')  # Initialize a quantum circuit with 4 qubits (We have 16 elements and 2^4 = 16)
  oracle.x(range(4))                         # Apply X gate to all 4 qubits turing |1111> into |0000>
  oracle.h(3)                                # Apply a Hadamard gate to the last qubit (3) to create a superposition state for controlled operations
  oracle.mcx([0, 1, 2], 3)                   # Apply a multi-controll X gate to flip the state of the target qubit (3) only if qubits 0, 1, and 2 are all in state |1>.
  oracle.h(3)                                # Apply another Hadamard gate to the last qubit to complete the marking process
  oracle.x(range(4))                         # Apply X gate to all 4 qubits again to revert them back to their original states
  oracle = oracle.to_gate()                  # Convert the quntum circuit into a gate so that it can be appended to other circuits
  return oracle


# Define a function to implement Grover's algorithm
def grover_circuit(n, oracle, iterations):
  '''This function constructs a quantum circuit implementing Grover's algorithm for an n-qubit system with a specified oracle and number of iterations.'''
  qc = QuantumCircuit(n)                   # Initialize a quantum circuit with n qubits
  qc.h(range(n))                           # Apply Hadamard gate to all qubits to create a superposition for all possible states
  # Perform the Grover iterations:
  for _ in range(iterations):
    qc.append(oracle, range(n))            # Apply the oracle to mark the solution state
    #Apply the Grover diffusion operator
    qc.h(range(n))
    qc.x(range(4))
    qc.h(n-1)
    qc.mcx(list(range(n-1)), n-1)
    qc.h(n-1)
    qc.x(range(n))
    qc.h(range(n))

  return qc


# Calculate the number of iterations
n = 4
N = 2**n
quantum_iterations = int(np.floor(np.pi / 4 * np.sqrt(N)))

# Create the Oracle and Grover circuit
oracle = grover_oracle()
grover = grover_circuit(4, oracle, quantum_iterations)

## Run using Simulator

We can implement the Grover's algoithm using the following Quantum simulator.

In [ ]:
# Transpile the circuit for the simulator backend
backend = Aer.get_backend('qasm_simulator')     # Quantum simulator backend by Qiskit
tqcs = transpile(grover, backend)

# Draw the circuit
print(circuit_drawer(tqcs, output='text'))

# Run the quantum circuit
result = backend.run(tqcs).result()

# Measure the runtime for the quantum apporach
quantum_runtime = result.time_taken

# Print the result
print(f"Quantum Search (Simulator): Time taken {quantum_runtime:.5f} seconds, number of iterations: {quantum_iterations}")


     ┌───┐┌───┐          ┌───┐┌───┐┌───┐               ┌───┐┌───┐┌───┐     »
q_0: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ X ├────────────■──┤ X ├┤ H ├┤ X ├─────»
     ├───┤├───┤       │  ├───┤├───┤├───┤            │  ├───┤├───┤├───┤     »
q_1: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ X ├────────────■──┤ X ├┤ H ├┤ X ├─────»
     ├───┤├───┤       │  ├───┤├───┤├───┤            │  ├───┤├───┤├───┤     »
q_2: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ X ├────────────■──┤ X ├┤ H ├┤ X ├─────»
     ├───┤├───┤┌───┐┌─┴─┐├───┤├───┤├───┤┌───┐┌───┐┌─┴─┐├───┤├───┤├───┤┌───┐»
q_3: ┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├»
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«               ┌───┐┌───┐┌───┐               ┌───┐┌───┐┌───┐               »
«q_0: ───────■──┤ X ├┤ H ├┤ X ├────────────■──┤ X ├┤ H ├┤ X ├────────────■──»
«            │  ├───┤├───┤├───┤            │  ├───┤├───┤├───┤            │  »
«q_1: ───────■──┤ X ├┤ H ├┤ X ├────────────■──┤ X ├┤ H ├┤ X ├────────────

## Run using IBM Quantum Computer

We can implement the Grover's algoithm using Quantum Computers at IBM through API calls

In [ ]:
# Replace myAPI value with your IBM Quantum API token generated from https://quantum.ibm.com/
myAPI = '078b489afdf77d18d7ce8ee8b9799ae82cb12ad3d85a981a6808424f78f553550f0d703af651e512ca8a340622f548a7df652d7a63ba140fb813eac1a3d85e50'

In [ ]:
# Load IBM Q account
IBMProvider.save_account(myAPI, overwrite=True)
provider = IBMProvider()

# Get the least busy backend device with at least 4 qubits
device = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 4 and not x.configuration().simulator and x.status().operational==True))
print(f"Running on current least busy device: {device}")

Running on current least busy device: <IBMBackend('ibm_osaka')>


In [ ]:
# Transpile the circuit for the selected backend device
tqcd = transpile(grover, device)

# Draw the circuit
print(circuit_drawer(tqcd, output='text'))

# Run the quantum circuit on the actual quantum device
job = device.run(tqcd)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Check the job status here: https://quantum.ibm.com/

In [ ]:
# Wait for the job to complete
while not job.done():
    print("Job is not done yet, checking again in 30 seconds...")
    time.sleep(30)

# Get the result
result = job.result()

# Measure the runtime for the quantum approach
quantum_runtime = result.time_taken

# Get the counts
counts = result.get_counts()

# Plot the histogram
plot_histogram(counts)
plt.show()

# Print the results
print(f"Quantum Search (Using IBM Quantum Device): Time taken {quantum_runtime:.5f} seconds, number of iterations {quantum_iterations}")

Quantum Search (Using IBM Quantum Device): Time taken 4.01711 seconds, number of iterations 3


# Explorations

From the code above and the search algorithms we can see that the quantum algorithm is definitely using less iterations compared to the classical search algorithm. However the runtime is not correct since we are not using a quantum computer directly. This stands as a proof to grover's algorithm's superiority over the classical search algorithm.

